In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [2]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)

#time["Elapsed time, mass 40"].describe()

       mass 2.0_value  mass 4.0_value  mass 5.5_value  mass 12.0_value  \
count    4.750000e+02    4.750000e+02    4.750000e+02     4.750000e+02   
mean     6.074262e-11    7.404977e-13    1.090263e-13     3.230105e-13   
std      5.055042e-12    1.499339e-13    2.920879e-14     1.572528e-14   
min      5.738620e-11    2.220000e-14    4.800000e-15     2.494000e-13   
25%      5.897690e-11    7.666500e-13    1.080000e-13     3.125500e-13   
50%      6.004630e-11    7.731000e-13    1.165000e-13     3.227000e-13   
75%      6.118575e-11    7.801500e-13    1.244500e-13     3.336500e-13   
max      9.361100e-11    8.057000e-13    1.537000e-13     3.648000e-13   

       mass 15.0_value  mass 16.0_value  mass 18.0_value  mass 28.0_value  \
count     4.750000e+02     4.750000e+02     4.750000e+02     4.750000e+02   
mean      1.894705e-13     6.075733e-13     1.350917e-12     1.924619e-12   
std       1.176345e-14     1.970131e-14     3.097804e-14     7.769109e-14   
min       1.021000e-13   

,mass 2.0_time,mass 4.0_time,mass 5.5_time,mass 12.0_time,mass 15.0_time,mass 16.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 40.0_time,...,"Elapsed time, mass 4","Elapsed time, mass 5.5","Elapsed time, mass 12","Elapsed time, mass 15","Elapsed time, mass 16","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 40","Elapsed time, mass 44"
2,2023/01/12 17:14:31.367,2023/01/12 17:14:34.618,2023/01/12 17:14:34.868,2023/01/12 17:14:35.118,2023/01/12 17:14:35.369,2023/01/12 17:14:35.619,2023/01/12 17:14:35.869,2023/01/12 17:14:36.119,2023/01/12 17:14:36.369,2023/01/12 17:14:37.371,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,NaN
3,2023/01/12 17:14:37.692,2023/01/12 17:14:40.943,2023/01/12 17:14:41.193,2023/01/12 17:14:41.443,2023/01/12 17:14:41.694,2023/01/12 17:14:41.944,2023/01/12 17:14:42.194,2023/01/12 17:14:42.444,2023/01/12 17:14:42.694,2023/01/12 17:14:42.945,...,6.325,6.325,6.325,6.325,6.325,6.325,6.325,6.325,5.574,NaN
4,2023/01/12 17:14:44.017,2023/01/12 17:14:47.268,2023/01/12 17:14:47.518,2023/01/12 17:14:47.768,2023/01/12 17:14:48.019,2023/01/12 17:14:48.269,2023/01/12 17:14:48.519,2023/01/12 17:14:48.769,2023/01/12 17:14:49.019,2023/01/12 17:14:49.270,...,12.650,12.650,12.650,12.650,12.650,12.650,12.650,12.650,11.899,NaN
5,2023/01/12 17:14:50.342,2023/01/12 17:14:53.593,2023/01/12 17:14:53.843,2023/01/12 17:14:54.093,2023/01/12 17:14:54.344,2023/01/12 17:14:54.594,2023/01/12 17:14:54.844,2023/01/12 17:14:55.094,2023/01/12 17:14:55.344,2023/01/12 17:14:55.595,...,18.975,18.975,18.975,18.975,18.975,18.975,18.975,18.975,18.224,NaN
6,2023/01/12 17:14:56.667,2023/01/12 17:14:59.918,2023/01/12 17:15:00.168,2023/01/12 17:15:00.418,2023/01/12 17:15:00.669,2023/01/12 17:15:00.919,2023/01/12 17:15:01.169,2023/01/12 17:15:01.419,2023/01/12 17:15:01.669,2023/01/12 17:15:01.920,...,25.300,25.300,25.300,25.300,25.300,25.300,25.300,25.300,24.549,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,2023/01/12 18:04:07.119,2023/01/12 18:04:07.369,2023/01/12 18:04:07.619,2023/01/12 18:04:07.869,2023/01/12 18:04:08.119,2023/01/12 18:04:08.370,2023/01/12 18:04:08.620,2023/01/12 18:04:08.871,2023/01/12 18:04:09.121,2023/01/12 18:04:09.371,...,2972.751,2972.751,2972.751,2972.750,2972.751,2972.751,2972.752,2972.752,2972.000,NaN
473,2023/01/12 18:04:13.444,2023/01/12 18:04:13.694,2023/01/12 18:04:13.944,2023/01/12 18:04:14.194,2023/01/12 18:04:14.444,2023/01/12 18:04:14.695,2023/01/12 18:04:14.945,2023/01/12 18:04:15.195,2023/01/12 18:04:15.446,2023/01/12 18:04:15.696,...,2979.076,2979.076,2979.076,2979.075,2979.076,2979.076,2979.076,2979.077,2978.325,NaN
474,2023/01/12 18:04:19.769,2023/01/12 18:04:20.019,2023/01/12 18:04:20.269,2023/01/12 18:04:20.519,2023/01/12 18:04:20.769,2023/01/12 18:04:21.020,2023/01/12 18:04:21.270,2023/01/12 18:04:21.520,2023/01/12 18:04:21.771,2023/01/12 18:04:22.021,...,2985.401,2985.401,2985.401,2985.400,2985.401,2985.401,2985.401,2985.402,2984.650,NaN
475,2023/01/12 18:04:26.094,2023/01/12 18:04:26.344,2023/01/12 18:04:26.594,2023/01/12 18:04:26.844,2023/01/12 18:04:27.094,2023/01/12 18:04:27.345,2023/01/12 18:04:27.595,2023/01/12 18:04:27.845,2023/01/12 18:04:28.096,2023/01/12 18:04:28.346,...,2991.726,2991.726,2991.726,2991.725,2991.726,2991.726,2991.726,2991.727,2990.975,NaN


In [3]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,12/01/2023 14:57:11,7.0,-0.014883,1.900000e-09,1.000000e-09,4.996000e-09,4.533000e-07,0.000572,235.0,-5.163,4249.702,NaN,NaN
2,NaN,12/01/2023 14:57:18,14.0,-0.015426,1.900000e-09,9.970000e-10,4.996000e-09,4.533000e-07,0.000571,235.0,-5.163,4250.039,NaN,NaN
3,NaN,12/01/2023 14:57:26,22.0,-0.014883,1.890000e-09,9.890000e-10,4.996000e-09,4.533000e-07,0.000606,235.0,-5.050,4250.264,NaN,NaN
4,NaN,12/01/2023 14:57:33,29.0,-0.017521,1.890000e-09,9.920000e-10,4.996000e-09,4.533000e-07,0.000569,234.0,-5.275,4250.039,NaN,NaN
5,NaN,12/01/2023 14:57:40,36.0,-0.018877,1.890000e-09,9.960000e-10,4.996000e-09,4.533000e-07,0.000569,235.0,-5.163,4250.488,NaN,NaN


In [25]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
#hv["em"]=abs(hv["em"])
#hv["hv_grid"]=abs(hv["hv_grid"])
hv["datetime"] = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
hv['datetime'] = hv['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')
#hv.insert(0,"datetime",hv["Date"]+" "+hv["Time"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)


hv.describe()




,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,IMon2,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,hv_grid,em
count,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.0,2.481000e+03,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2481.000000,2.481000e+03,2.481000e+03
mean,1487.791979,212.279339,72.953027,728.300032,728.463926,723.878210,363.743700,0.0,4.000000e-02,-0.114518,0.859706,7.470603,7.488840,7.456813,7.469833,-0.018642,-0.012017,-1.145177e-07,8.597060e-07
std,861.175741,245.965490,44.432670,443.768193,443.703064,445.932542,222.301784,0.0,1.742013e-15,0.199254,1.298677,4.615580,4.622695,4.614935,4.610082,0.002558,0.001548,1.992544e-07,1.298677e-06
min,0.000000,0.000000,0.060000,0.000000,0.000000,0.000000,0.000000,0.0,4.000000e-02,-0.767000,-3.783400,-3.089600,-2.613500,-1.718200,-0.643600,-0.022600,-0.015900,-7.670000e-07,-3.783400e-06
25%,741.750000,0.060000,0.120000,0.220000,0.200000,0.100000,0.060000,0.0,4.000000e-02,-0.385700,0.015100,0.025200,0.044800,0.025600,0.043500,-0.021000,-0.012600,-3.857000e-07,1.510000e-08
50%,1487.534000,0.160000,100.060000,1000.000000,1000.120000,1000.080000,499.960000,0.0,4.000000e-02,-0.001600,0.023100,10.048600,10.067500,10.031800,10.044000,-0.018800,-0.011700,-1.600000e-09,2.310000e-08
75%,2233.819000,499.940000,100.080000,1000.020000,1000.160000,1000.100000,499.980000,0.0,4.000000e-02,0.018600,2.628900,10.798600,10.820500,10.785500,10.795300,-0.017300,-0.010900,1.860000e-08,2.628900e-06
max,2973.988000,499.980000,100.180000,1000.080000,1000.200000,1000.220000,500.100000,0.0,4.000000e-02,0.529700,3.847100,11.187000,11.206700,11.171800,11.179300,-0.013200,-0.009800,5.297000e-07,3.847100e-06


### Injection pressure linearity

In [5]:
#defining a function for the straight line
def func(x,a,b):
    return a*x + b
#curve fit for the data
y = vaclog["injection 1mbar"].values
x = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(func,x,y)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.02*x+0.26


In [6]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = func(x,a,b)
ax.plot((vaclog["Elapsed time"]/3600),vaclog["injection 1mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.plot((vaclog["Elapsed time"]/3600),y_fit, "--",color="red", label="Y= -ax + b fit") 
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 1mbar conductance gauge")

Text(0.5, 1.0, 'Injection pressure evolution 1mbar conductance gauge')

### Data processing

In [7]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [8]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
print("Result 1:",hv.loc[hv["Comment"]=="grid transparent"])
print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
print("Result 3:", hv.loc[hv["Comment"]=="electron activity grid repelling"])

x=pd.Timestamp(2023, 1, 12, 17 ,23, 58)
y=pd.Timestamp(2023, 1, 12, 17, 28, 43)
z=pd.Timestamp(2023, 1, 12, 17, 20, 13)
#then find the time value closest to it in the MID scan file 
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
print("\n","Result 5:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])
print("\n","Result 6:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - z).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 40.0_value"])

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck

 
#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][91:136]/h2_sens
print(p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][56:91]/h2_sens
base_h2.describe()

Result 1:             Date          Time  Elapsed time   VMon1   VMon2    VMon3  \
481   12/01/2023  17:23:58.050       575.690    0.30  100.00   999.98   
482   12/01/2023  17:23:59.249       576.889    0.24  100.02   999.98   
483   12/01/2023  17:24:00.477       578.117    0.18  100.04   999.98   
484   12/01/2023  17:24:01.678       579.318    0.18  100.04   999.98   
819   12/01/2023  17:30:43.081       980.721  219.06  100.08  1000.00   
820   12/01/2023  17:30:44.276       981.916   88.10  100.00  1000.04   
821   12/01/2023  17:30:45.471       983.111    0.90  100.00  1000.02   
822   12/01/2023  17:30:46.669       984.309    0.66  100.02  1000.02   
1287  12/01/2023  17:40:07.180      1544.820    0.16  100.00  1000.02   
1288  12/01/2023  17:40:08.373      1546.013    0.16  100.02  1000.02   
1289  12/01/2023  17:40:09.602      1547.242    0.16  100.02  1000.02   
1290  12/01/2023  17:40:10.808      1548.448    0.16  100.02  1000.02   
1291  12/01/2023  17:40:12.011      1549.


 Result 6:                mass 2.0_time  mass 2.0_value            mass 4.0_time  \
56   2023/01/12 17:20:15.919    6.143140e-11  2023/01/12 17:20:16.169   
55   2023/01/12 17:20:09.594    6.136210e-11  2023/01/12 17:20:09.844   
57   2023/01/12 17:20:22.244    6.106590e-11  2023/01/12 17:20:22.494   
54   2023/01/12 17:20:03.269    6.157120e-11  2023/01/12 17:20:03.519   
58   2023/01/12 17:20:28.569    6.128950e-11  2023/01/12 17:20:28.819   
..                       ...             ...                      ...   
472  2023/01/12 18:04:07.119    5.774840e-11  2023/01/12 18:04:07.369   
473  2023/01/12 18:04:13.444    5.801880e-11  2023/01/12 18:04:13.694   
474  2023/01/12 18:04:19.769    5.815160e-11  2023/01/12 18:04:20.019   
475  2023/01/12 18:04:26.094    5.772000e-11  2023/01/12 18:04:26.344   
476  2023/01/12 18:04:32.419    5.771570e-11  2023/01/12 18:04:32.669   

     mass 4.0_value            mass 5.5_time  mass 5.5_value  \
56     7.918000e-13  2023/01/12 17:20:16.419   

count    3.500000e+01
mean     2.879128e-09
std      2.774300e-11
min      2.859813e-09
25%      2.866385e-09
50%      2.873160e-09
75%      2.879266e-09
max      3.004606e-09
Name: mass 2, corr, dtype: float64

## Point-by-point desorption

In [9]:
#make a sub-df from mid that includes only mass 2 data
h2 = mid.loc[:, ["mass 2.0_time", "mass 2.0_value"]]
#rename columns
h2 = h2.rename(columns={"mass 2.0_time": "datetime","mass 2.0_value": "Ion current"}).reset_index(drop=True)

print(h2)

                    datetime   Ion current
0    2023/01/12 17:14:31.367  9.361100e-11
1    2023/01/12 17:14:37.692  9.281010e-11
2    2023/01/12 17:14:44.017  9.198780e-11
3    2023/01/12 17:14:50.342  9.152550e-11
4    2023/01/12 17:14:56.667  9.081060e-11
..                       ...           ...
470  2023/01/12 18:04:07.119  5.774840e-11
471  2023/01/12 18:04:13.444  5.801880e-11
472  2023/01/12 18:04:19.769  5.815160e-11
473  2023/01/12 18:04:26.094  5.772000e-11
474  2023/01/12 18:04:32.419  5.771570e-11

[475 rows x 2 columns]


In [10]:
# Convert the 'time' column to datetime index for both dataframes
hv.index = (pd.to_datetime(hv['datetime']).dt.round(freq="S"))
h2.index = (pd.to_datetime(h2['datetime']).dt.round(freq="S"))
print(hv)

# Reindex df1 to match the datetime index of df2
hv = hv.reindex(h2.index, method='nearest', limit=1)

#Drop the "datetime" columns
hv.drop('datetime', axis=1, inplace=True)
h2.drop('datetime', axis=1, inplace=True)

# Now df1 and df2 have the same datetime index and can be joined or aligned
# by joining on the index
hv_aligned = hv.merge(h2,left_index=True, right_index=True, how="inner")

print("aligned",hv_aligned)
print(hv_aligned.describe())

                           Date          Time  Elapsed time  VMon1  VMon2  \
datetime                                                                    
2023-01-12 17:14:22  12/01/2023  17:14:22.360         0.000   0.00   0.08   
2023-01-12 17:14:24  12/01/2023  17:14:23.542         1.182   0.00   0.08   
2023-01-12 17:14:25  12/01/2023  17:14:24.726         2.366   0.00   0.08   
2023-01-12 17:14:26  12/01/2023  17:14:25.906         3.546   0.00   0.08   
2023-01-12 17:14:27  12/01/2023  17:14:27.079         4.719   0.00   0.08   
...                         ...           ...           ...    ...    ...   
2023-01-12 18:03:52  12/01/2023  18:03:51.622      2969.262   0.06   0.08   
2023-01-12 18:03:53  12/01/2023  18:03:52.798      2970.438   0.06   0.08   
2023-01-12 18:03:54  12/01/2023  18:03:53.983      2971.623   0.06   0.08   
2023-01-12 18:03:55  12/01/2023  18:03:55.164      2972.804   0.06   0.08   
2023-01-12 18:03:56  12/01/2023  18:03:56.348      2973.988   0.06   0.08   

In [11]:
#Filtering background signal for interpolated datasets

#take a mass with a signal known to be in the background
bck = np.mean(mid["mass 5.5_value"])

#apply background correction to the rest of the mass spectrum
hv_aligned["mass 2 corr"]=hv_aligned["Ion current"]-bck

#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #A/mbar
p_h2_ip = hv_aligned["mass 2 corr"]/(h2_sens*0.46) #H2 equivalent pressures

#account for the thermal transpiration factor
T2 = 4.2
T1 = 296.5
T_coef = np.sqrt(T2/T1)
p_h2_ip = p_h2_ip*T_coef 

#insert partial pressures into the dataframe
hv_aligned.insert(22,"hydrogen pp",p_h2_ip)


In [12]:
hv_aligned.fillna(0,inplace=True)
hv_aligned["hydrogen pp"].describe()

count    4.750000e+02
mean     7.365261e-10
std      6.140442e-11
min      6.957552e-10
25%      7.150777e-10
50%      7.280678e-10
75%      7.419089e-10
max      1.135784e-09
Name: hydrogen pp, dtype: float64

# Primary desorption calculation method
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.

Due to the EGA being fully retracted, it is safe to assume that the desorption takes place at approximately room temperature

In [13]:
#calculation delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = 293
print(T)
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)
#Emission current

I_em = np.nanmean(hv["em"].nlargest(30))
print(I_em)
#Conductance for CO in m3/s
c_co=10.13/1000
c_co2 = 8.08/1000


2.065057008718916e-09
293
0.01992516581153798
3.0179133333333332e-06


In [14]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

The desorption yield for H2 [mol/el]: 0.0005395112457673308 



## Desorption yield point-by-point

$$
\eta=\frac{C_{H2}\cdot (p_{n} - p_{n-1}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$


In [15]:
#partial pressure delta

p_delta = hv_aligned["hydrogen pp"].diff(periods=5).fillna(hv_aligned["hydrogen pp"])*100
p_delta = p_delta.clip(lower=0)
print(p_delta)
#hv_shifted = hv_aligned.shift(periods=1)
#print(hv_shifted)
#p_delta = hv_aligned["hydrogen pp"] - hv_shifted["hydrogen pp"]
#print(hv_aligned["hydrogen pp"].describe())
#print(hv_shifted["hydrogen pp"].describe())
print(p_delta.describe())

datetime
2023-01-12 17:14:31    1.135784e-07
2023-01-12 17:14:38    1.126055e-07
2023-01-12 17:14:44    1.116066e-07
2023-01-12 17:14:50    1.110451e-07
2023-01-12 17:14:57    1.101767e-07
                           ...     
2023-01-12 18:04:07    4.399702e-10
2023-01-12 18:04:13    0.000000e+00
2023-01-12 18:04:20    4.466512e-10
2023-01-12 18:04:26    0.000000e+00
2023-01-12 18:04:32    2.448868e-10
Name: hydrogen pp, Length: 475, dtype: float64
count    4.750000e+02
mean     1.347887e-09
std      1.141198e-08
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.952656e-10
max      1.135784e-07
Name: hydrogen pp, dtype: float64


In [16]:
#constants are already defined
#emission current and H2 partial pressures are now available at the same sampling rates
print(hv_aligned.describe())
I_n = hv_aligned["em"].replace(0, np.nan).dropna()

I_n.where(hv_aligned["em"] >= 8e-7, np.nan, inplace=True)
#pd.df.where(cond= "Where cond is True, keep the original value. Otherwise, replace with NaN")
print(I_n.describe())

eta = ((x_h2*p_delta*q_e)/(k_B*T*I_n)).dropna()
eta = eta.loc[eta.index[1]:]
print("desorption yields point-by-point:","\n", eta)

       Elapsed time       VMon1       VMon2        VMon3        VMon4  \
count    475.000000  475.000000  475.000000   475.000000   475.000000   
mean    1479.487358  211.152884   72.494737   723.813558   724.020674   
std      868.182625  245.777389   44.726711   446.155768   446.043826   
min        0.000000    0.000000    0.000000     0.000000     0.000000   
25%      729.199500    0.060000    0.120000     0.220000     0.180000   
50%     1480.319000    0.160000  100.060000  1000.000000  1000.120000   
75%     2228.405000  499.940000  100.080000  1000.020000  1000.160000   
max     2973.988000  499.980000  100.140000  1000.040000  1000.180000   

             VMon5       VMon6  VMon7       VMon8       IMon1  ...  \
count   475.000000  475.000000  475.0  475.000000  475.000000  ...   
mean    719.565811  361.598653    0.0    0.039579   -0.112787  ...   
std     448.408449  223.478666    0.0    0.004087    0.200556  ...   
min       0.000000    0.000000    0.0    0.000000   -0.506800 

In [17]:
#slicing eta to match hv_aligned["Elapsed time"]:
# Slice x using the index of y
hv_downsampled = hv_aligned.loc[eta.index, :]

# Plotting

In [18]:
#plot the aligned dataframe results
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.plot(hv_aligned["Elapsed time"],hv_aligned["em"],linestyle="-",marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [19]:
#plot the h2 mid 
#seems to be too little rows?
plt.figure(figsize=(12,6))
plt.plot(hv_aligned["Elapsed time"],hv_aligned["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(loc="upper right")
plt.title("Ion current evolution of H2, 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('linear')


In [20]:
#Plotting the all the relevant MID spectrums
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")

plt.yscale('log')

In [21]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")


plt.yscale('log')


In [22]:
#Plotting BA2, BA1 pressure
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

In [23]:
## Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Elapsed time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("Temperature evolution 1 ML tech. surface pre-injection, EGA fully in, dynamic vacuum")
plt.yscale('linear')

In [24]:
#plotting point-by-point eta
plt.figure(figsize=(10,6))
plt.plot(hv_downsampled["Elapsed time"],eta,marker=".", markersize=4,label='eta')
#plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Electon stimulated desorption yield (M/el)')
plt.legend()
plt.title("ESD vs time, p_n - p_n-5")
ax = plt.gca()
ax.set_yscale('log')
ax.grid(which="major",axis="x")
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()